In [4]:
from lively import Solver, Translation, Rotation, SmoothnessMacroObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

# Constructing Collision Avoidance objective and box shape
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation=Translation(x=0.0,y=0.0,z=0.0), rotation=Rotation(x=0.0,y=0.0,z=0.0,w=1.0))

# print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objectives
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=2.5,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve({},{},0.0)
# Log the initial state
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'z': 0.0, 'y': 2.5, 'x': 0.0}, {'w': 1.0, 'y': 0.0, 'x': 0.0, 'z': 0.0})
{'panda_joint1': -2.1509353916306086e-07, 'panda_joint3': -2.1449723173547139e-07, 'panda_joint7': -0.00012837884160520904, 'panda_joint2': -3.560324793054898e-06, 'panda_joint6': 1.867502227751826, 'panda_joint5': -4.066489393808552e-08, 'panda_joint4': -1.5708350398207753, 'panda_finger_joint1': 0.02898657198620339, 'panda_finger_joint2': 0.02898657198620339}
TransformInfo: world:{translation: {x:0.0, y:2.5, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:2.5, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999998841, points: Some(([-2.0843863685980674e-7, 2.5000000000001226, 0.3929999999996318], [-4.078409865346955e-8, 2.5000000000000906, 0.34299999999992453])), physical: true, loss: 1.0499999999999885, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_link5", d